In [1]:
'''
All the data are fetched from tmdb api, TMDB is same as IMDB and the database has both movies and tv series.
For this project i have fetched 10K most popular movies available on TMDB database. TMDB has a page limit of 500 hence only 10K
data was extracted.
The fetched data was converted to dataframe and exported into csv files that are to be used for building the project.
'''  


'\nAll the data are fetched from tmdb api, TMDB is same as IMDB and the database has both movies and tv series.\nFor this project i have fetched 10K most popular movies available on TMDB database. TMDB has a page limit of 500 hence only 10K\ndata was extracted.\nThe fetched data was converted to dataframe and exported into csv files that are to be used for building the project.\n'

In [2]:
import requests
import pandas as pd

# Popular Movies

In [3]:
api_key = 'ca429b00e545815aa26d6439d0e39f12'

In [4]:
movies = pd.DataFrame()
for i in range(1,501):
    url = "https://api.themoviedb.org/3/movie/popular?api_key={}&language=en-US&page={}".format(api_key,i)

    response = requests.get(url)

    response = response.json()
    movies = pd.concat([movies,pd.DataFrame(response['results'])])

movies.shape

(10000, 14)

In [5]:
movies.head(2)

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/cVh8Af7a9JMOJl75ML3Dg2QVEuq.jpg,"[12, 10751, 16]",762509,en,Mufasa: The Lion King,"Mufasa, a cub lost and alone, meets a sympathe...",3353.171,/9bXHaLlsFYpJUutg4E6WXAjaxDi.jpg,2024-12-18,Mufasa: The Lion King,False,7.462,1178
1,False,/zo8CIjJ2nfNOevqNajwMRO6Hwka.jpg,"[16, 12, 10751, 35]",1241982,en,Moana 2,After receiving an unexpected call from her wa...,2296.395,/aLVkiINlIeCkcZIzb7XHzPYgO6L.jpg,2024-11-21,Moana 2,False,7.200,1605


Genres

In [6]:
# in movies dataframe we have genre_id but we need genres hence another response is requested from that returns the id and name 
# of genre

url = 'https://api.themoviedb.org/3/genre/movie/list?api_key={}'.format(api_key)
response = requests.get(url).json()['genres']

response

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 80, 'name': 'Crime'},
 {'id': 99, 'name': 'Documentary'},
 {'id': 18, 'name': 'Drama'},
 {'id': 10751, 'name': 'Family'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 36, 'name': 'History'},
 {'id': 27, 'name': 'Horror'},
 {'id': 10402, 'name': 'Music'},
 {'id': 9648, 'name': 'Mystery'},
 {'id': 10749, 'name': 'Romance'},
 {'id': 878, 'name': 'Science Fiction'},
 {'id': 10770, 'name': 'TV Movie'},
 {'id': 53, 'name': 'Thriller'},
 {'id': 10752, 'name': 'War'},
 {'id': 37, 'name': 'Western'}]

In [7]:
genre_dict = {genre['id']:genre['name'] for genre in response}
genre_dict

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [8]:
movies['genre'] = movies['genre_ids'].apply(lambda x: [genre_dict[id] for id in x])
movies.drop(columns = ['genre_ids'],inplace = True)
movies.to_csv('Movies 10k.csv',index = False)

Credit

In [9]:
# this url gives the crew and cast for the movies

In [10]:
id = movies['id'].tolist()

In [11]:
credit = pd.DataFrame()

for i in id:
    url = "https://api.themoviedb.org/3/movie/{}/credits?api_key={}&language=en-US".format(i,api_key)
    response = requests.get(url)
    response = response.json()
    df = pd.DataFrame().from_dict(response,orient = 'index').T
    credit = pd.concat([credit,df])

credit.shape

(10000, 3)

In [12]:
credit.to_csv('Credit 10k.csv',index = False)

Keywords

In [13]:
# the main keywords of the movie
keyword = pd.DataFrame()
for i in id:
    url = "https://api.themoviedb.org/3/movie/{}/keywords?api_key={}".format(i,api_key)
    response = requests.get(url)
    response = response.json()
    keyword = pd.concat([keyword,pd.DataFrame([response])])

keyword.shape  

(10000, 2)

In [14]:
keyword.to_csv('Keywords.csv',index = False)

Poster

In [16]:
# I want to show posters along with the name of recommended movie. first we had to extract the image url and then the poster url
# the final url consist of image url, poster size, and poster path

configuration = requests.get('https://api.themoviedb.org/3/configuration?api_key={}'.format(api_key))
configuration = configuration.json()

image_base_url = configuration['images']['base_url']
poster_size = 'w500'

response = requests.get('https://api.themoviedb.org/3/movie/{}/images?api_key={}&language=en'.format(i,api_key))

image =[]
for i in id:
    response = requests.get('https://api.themoviedb.org/3/movie/{}/images?api_key={}&include_image_language=en%2Cnull?'.format(i,api_key))
    if response.json()['posters'] != []:
        file_path = response.json()['posters'][0]['file_path']
        image_path = image_base_url+poster_size+file_path
        image.append([i,image_path])
    
image_url = pd.DataFrame(image, columns = ['id','image_url'])
image_url.shape

(9180, 2)

In [17]:
image_url.to_csv('image_url.csv',index = False)

In [18]:
image_url['image_url'].iloc[0]

'http://image.tmdb.org/t/p/w500/9bXHaLlsFYpJUutg4E6WXAjaxDi.jpg'

In [19]:
image_base_url + '/w500/tkG0qgMErYsxZ5aHcW7m55yfSIj.jpg'

'http://image.tmdb.org/t/p//w500/tkG0qgMErYsxZ5aHcW7m55yfSIj.jpg'